In [1]:
import pandas as pd 

In [2]:
import pandas as pd
import os

file_path = (r'C:\Users\Pedro Soares\Desktop\portfolio\aguiabranca\data\data\acidentes_mg.csv')

if os.path.exists(file_path):
    try:
        df = pd.read_csv(file_path, sep=';', encoding='latin-1')
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, sep=';', encoding='cp1252')
    
    print(df.head(2))


         data   horario  n_da_ocorrencia tipo_de_ocorrencia     km     trecho  \
0  22/09/2022  12:10:00               20         02 - Acide   91,1  Trecho 03   
1  22/09/2022  14:46:00               27         02 - Acide  143,9  Trecho 02   

  sentido  tipo_de_acidente  automovel  bicicleta  ...  outros  tracao_animal  \
0     Sul  Colisão Traseira          1          0  ...       0              0   
1     Sul  Colisão Traseira          1          0  ...       0              0   

   transporte_de_cargas_especiais  trator_maquinas  utilitarios  ilesos  \
0                               0                0            0       0   
1                               0                0            0       1   

   levemente_feridos  moderadamente_feridos  gravemente_feridos  mortos  
0                  2                      0                   0       0  
1                  2                      0                   0       0  

[2 rows x 23 columns]


In [3]:
df.columns

Index(['data', 'horario', 'n_da_ocorrencia', 'tipo_de_ocorrencia', 'km',
       'trecho', 'sentido', 'tipo_de_acidente', 'automovel', 'bicicleta',
       'caminhao', 'moto', 'onibus', 'outros', 'tracao_animal',
       'transporte_de_cargas_especiais', 'trator_maquinas', 'utilitarios',
       'ilesos', 'levemente_feridos', 'moderadamente_feridos',
       'gravemente_feridos', 'mortos'],
      dtype='object')

### Irei remover a coluna "Bicicletas" pra não dar ruído no modelo visto que foi um baixo número.

In [4]:
df.drop(columns=['bicicleta'], inplace=True)

#### Criando uma Modelagem Dimensional para tabelas e consultas precisas.

In [5]:
fato_acidentes = df[['n_da_ocorrencia','data','trecho','km', 'tipo_de_ocorrencia',
                     'tipo_de_acidente', 'automovel', 'moto', 'caminhao', 'onibus','outros'
                     ,'mortos']].copy()

In [6]:
### Criando dimensão tempo  ###

df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
dim_tempo = pd.DataFrame({'data': df['data'],
                          'ano': df['data'].dt.year,
                          'mes': df['data'].dt.month,
                          'dia': df['data'].dt.day,
                          'dia_da_semana': df['data'].dt.day_name()})

In [7]:
### criando dimensão veiculos ###

dim_veiculo = pd.DataFrame({'id_veiculo':[1,2,3], 
                            'tipo_veiculo':['Automóvel','Bicicleta','Moto']})




In [8]:
dim_classificacao = df[['tipo_de_ocorrencia','tipo_de_acidente']].drop_duplicates().reset_index(drop=True)

In [9]:
dim_tempo.head()

,data,ano,mes,dia,dia_da_semana
0,2022-09-22,2022,9,22,Thursday
1,2022-09-22,2022,9,22,Thursday
2,2022-09-23,2022,9,23,Friday
3,2022-09-24,2022,9,24,Saturday
4,2022-09-25,2022,9,25,Sunday


In [10]:
# CRIANDO FILTRO PARA DIA ESPECÍFICO
df_dia_especifico = dim_tempo[(dim_tempo['dia'].between (22, 30)) & 
                               (dim_tempo['mes'] == 9) & 
                               (dim_tempo['ano'] == 2022)]

# Contar quantos acidentes houve nesse dia
total_acidentes = len(df_dia_especifico)
print(f"Total de acidentes em 22/09/2022: {total_acidentes}")


Total de acidentes em 22/09/2022: 18


In [11]:
df.dtypes

data                              datetime64[ns]
horario                                   object
n_da_ocorrencia                            int64
tipo_de_ocorrencia                        object
km                                        object
trecho                                    object
sentido                                   object
tipo_de_acidente                          object
automovel                                  int64
caminhao                                   int64
moto                                       int64
onibus                                     int64
outros                                     int64
tracao_animal                              int64
transporte_de_cargas_especiais             int64
trator_maquinas                            int64
utilitarios                                int64
ilesos                                     int64
levemente_feridos                          int64
moderadamente_feridos                      int64
gravemente_feridos  

In [12]:
total_por_ano = df.groupby(df['data'].dt.year)['levemente_feridos'].sum()
print(total_por_ano[2022])


94


In [13]:
total_registros_2022 = len(df[df['data'].dt.year == 2022])
print(total_registros_2022)


301
